In [28]:
import pandas as pd
import pandahouse as ph
import random
from scipy import stats
import matplotlib.pyplot as plt
connection = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'simulator_20220720',
                      'user':'student', 
                      'password':''
                     }


def break_into_groups(n, connection):  #  создать датафреймы для каждой из групп
    q = """
    SELECT exp_group, 
        user_id,
        sum(action = 'like') as likes,
        sum(action = 'view') as views,
        likes/views as ctr
    FROM {db}.feed_actions 
    WHERE toDate(time) between '2022-06-13' and '2022-06-19' 
        and exp_group==%s
    GROUP BY exp_group, user_id
    """ % n
    df = ph.read_clickhouse(q, connection=connection)
    return df
'''
def add_CTR(df, CTR_c, likes_count):
    new_metric=[]
    for i in range(1, likes_count):
         new_metric[i]=df.likes[i]-𝐶𝑇𝑅_c∗df.views[i]
    return new_metric
'''


def take_a_test(num_1, num_2):  #  функция проведение тестов, подавть на вход первым номер контрольной группы
    df_1=break_into_groups(num_1, connection)
    df_2=break_into_groups(num_2, connection)
    𝐶𝑇𝑅𝑐𝑜𝑛𝑡𝑟𝑜𝑙_1=𝑠𝑢𝑚(df_1.𝑙𝑖𝑘𝑒𝑠)/𝑠𝑢𝑚(df_1.𝑣𝑖𝑒𝑤𝑠)
    
    df_1["linearized_likes"] = df_1.likes - 𝐶𝑇𝑅𝑐𝑜𝑛𝑡𝑟𝑜𝑙_1 *df_1.views
    df_2["linearized_likes"] = df_2.likes - 𝐶𝑇𝑅𝑐𝑜𝑛𝑡𝑟𝑜𝑙_1 *df_2.views
    
    test_before = stats.ttest_ind(df_1.ctr, df_2.ctr, equal_var=False)
    test_after = stats.ttest_ind(df_1.linearized_likes, df_2.linearized_likes, equal_var=False)
    return test_before, test_after


test_before_03, test_after_03 =  take_a_test(0, 3)  #  тест между группами 0 и 3
print('Результаты до преобразования для 0 и 3:',test_before_03)
print('Результаты после для 0 и 3:', test_after_03)

test_before_12, test_after_12 =  take_a_test(1, 2)  #  тест между группами 1 и 2
print('Результаты до преобразования для 1 и 2:',test_before_12)
print('Результаты после для 1 и 2:', test_after_12)
    


Результаты до преобразования для 0 и 3: Ttest_indResult(statistic=-13.896870721904069, pvalue=1.055849414662529e-43)
Результаты после для 0 и 3: Ttest_indResult(statistic=-15.21499546090383, pvalue=5.4914249479687664e-52)
Результаты до преобразования для 1 и 2: Ttest_indResult(statistic=0.7094392041270486, pvalue=0.4780623130874935)
Результаты после для 1 и 2: Ttest_indResult(statistic=6.122579994775972, pvalue=9.439432187037712e-10)


Для 0 и 3: 𝑝−𝑣𝑎𝑙𝑢𝑒 уменьшилось 

Для 1 и 2: 𝑝−𝑣𝑎𝑙𝑢𝑒 уменьшилось 